# Aim: 
### To create a program to simulaneously read both LULC and LSt images and calculate average temperatures of only those pixels marked as urban


### Import

In [2]:
import os
import re
import numpy as np
import pandas as pd
#from matplotlib import pyplot as plt
from osgeo import gdal

### Functions

In [3]:
path_lst  = r"E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\LST_baroda"
path_lulc = r"E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\LULC_baroda"
path_ndvi = r"E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\NDVI_Barod"
path_ndbi = r"E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\NDBI_Barod"

In [4]:
lst_list=[]  #init list of lst 
lulc_list=[] #init list of lulc
ndvi_list=[]
ndbi_list=[]

df = pd.DataFrame()
dic = dict()
#make list of lst images
for i in os.listdir(path_lst):
    if i.endswith(".tif"):
        lst_list.append(i)
#make list of lulc images        
for i in os.listdir(path_lulc):
    if i.endswith(".tif"):
        lulc_list.append(i)        
#make list of lulc images        
for i in os.listdir(path_ndvi):
    if i.endswith(".tif"):
        ndvi_list.append(i)        
#make list of lulc images        
for i in os.listdir(path_ndbi):
    if i.endswith(".tif"):
        ndbi_list.append(i)    

In [ ]:
for (lstFile, lulcFile, ndviFile, ndbiFile) in zip(lst_list, lulc_list, ndvi_list, ndbi_list):
    #open files individually with GDAL
    lst_ds = gdal.Open(path_lst+ "/" + lstFile)
    lulc_ds = gdal.Open(path_lulc+ "/" + lulcFile)
    ndvi_ds = gdal.Open(path_ndvi+ "/" + ndviFile)
    ndbi_ds = gdal.Open(path_ndbi+ "/" + ndbiFile)
    #get band 1 for each obtained file
    lst_band =  lst_ds.GetRasterBand(1)
    lulc_band = lulc_ds.GetRasterBand(1)
    ndvi_band = ndvi_ds.GetRasterBand(1)
    ndbi_band = ndbi_ds.GetRasterBand(1)
    
    #lst_ds ,lulc_ds , ndvi_ds , ndbi_ds = None,None,None,None
    
    yr = int((re.search( r"\d{4}" , lstFile )).group())
    
    #print(gdal.GetDataTypeName(band.DataType)  )
    #read raster as 1D array
    LstArray = lst_band.ReadAsArray().flatten()
    LulcArray = lulc_band.ReadAsArray().flatten()
    ndviArray = ndvi_band.ReadAsArray().flatten()
    ndbiArray = ndbi_band.ReadAsArray().flatten()
    
    #counters of no of pixel of each LU
    counter_u=0 # counts the number of pixels with urban classification
    counter_g=0 #green
    counter_w=0 #water
    counter_b=0 #barren
    lt_u, lt_g, lt_w , lt_b = (0,0,0,0)
    ndv_u, ndv_g, ndv_w , ndv_b = (0,0,0,0)
    ndb_u, ndb_g, ndb_w , ndb_b = (0,0,0,0)
    
    for (lulc, lst, ndv, ndb) in zip( LulcArray , LstArray ,  ndviArray , ndbiArray ):
        if lulc==1:
            lt_u += lst
            ndv_u += ndv
            ndb_u += ndb
            counter_u += 1

        elif(lulc==2):
            lt_g += lst
            ndv_g += ndv
            ndb_g += ndb
            counter_g += 1

        elif(lulc==3):
            lt_w += lst
            ndv_w += ndv
            ndb_w += ndb
            counter_w += 1

        elif(lulc==4):
            lt_b += lst
            ndv_b += ndv
            ndb_b += ndb
            counter_b += 1

    avg_lst_u = lt_u/counter_u
    avg_lst_g = lt_g/counter_g
    avg_lst_w = lt_w/counter_w
    avg_lst_b = lt_b/counter_b
    
    avg_ndbi_u = ndb_u/counter_u
    avg_ndbi_g = ndb_g/counter_g
    avg_ndbi_w = ndb_w/counter_w
    avg_ndbi_b = ndb_b/counter_b
    
    avg_ndvi_u = ndv_u/counter_u
    avg_ndvi_g = ndv_g/counter_g
    avg_ndvi_w = ndv_w/counter_w
    avg_ndvi_b = ndv_b/counter_b
    
    avg_lst_tot= (lt_u+lt_g+lt_w+lt_b)/(counter_u+counter_b+counter_w+counter_g)
    avg_ndvi_tot= (ndv_u+ndv_g+ndv_w+ndv_b)/(counter_u+counter_b+counter_w+counter_g)
    avg_ndbi_tot= (ndb_u+ndb_g+ndb_w+ndb_b)/(counter_u+counter_b+counter_w+counter_g)
    
    #print(lstFile, "  " , lulcFile)
    dic[yr] = {'LST_urban': avg_lst_u, 'LST_green':avg_lst_g, 'LST_water':avg_lst_w, 'LST_barren':avg_lst_b , 
               'NDVI_urban': avg_ndvi_u, 'NDVI_green':avg_ndvi_g, 'NDVI_water':avg_ndvi_w, 'NDVI_barren':avg_ndvi_b , 
               'NDBI_urban': avg_ndbi_u, 'NDBI_green':avg_ndbi_g, 'NDBI_water':avg_ndbi_w, 'NDBI_barren':avg_ndbi_b ,
               'LST_avg': avg_lst_tot , 'NDVI_avg': avg_ndvi_tot, 'NDBI_avg': avg_ndbi_tot
              }
    
    x = pd.DataFrame() # dataframe for individual year
    x['Year'] = [yr for i in range(len(LulcArray))]
    x['LULC'] = LulcArray
    x['LST']  = LstArray
    x['NDVI'] = ndviArray
    x['NDBI'] = ndbiArray
    x = x.dropna().reset_index().drop(['index'],axis=1)
    
    df=df.append(x , ignore_index=True)
    print(yr , " : lst : ", avg_lst_u)
    #df[yr] = avg_lst_u
    
    
#df_LST = df.T
print("Tadaa")

In [ ]:
"""df"""

In [ ]:
df.to_json(r'E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\pixeldat.json')
imagedf = pd.DataFrame(dic).T
imagedf.to_json(r'E:\SPAD\4.Fourth_Year\08Eighth_Semester\help_given\self\paper\cities\new\Vodadara\imagedat.json')


# Plotting

In [99]:
"""%matplotlib inline
import plotly.express as px

fig = px.scatter_matrix(df,
    dimensions=['LST','NDVI','NDBI'],
    color="LULC")
fig.show()"""

ValueError: Value of 'dimensions_0' is not the name of a column in 'data_frame'. Expected one of ['LST_urban', 'LST_green', 'LST_water', 'LST_barren', 'NDVI_urban', 'NDVI_green', 'NDVI_water', 'NDVI_barren', 'NDBI_urban', 'NDBI_green', 'NDBI_water', 'NDBI_barren', 'LST_avg', 'NDVI_avg', 'NDBI_avg'] but received: LST

### Export to excel

In [89]:
"""# exporting to excel
writer = pd.ExcelWriter('data_vadodara.xlsx', engine='openpyxl')
#df.to_excel(writer, 'pixel_data')
#writer.save()   # This works
df1.to_excel(writer, 'image_data')
writer.save()   # This works too!  """  

ValueError: This sheet is too large! Your sheet size is: 2357600, 5 Max sheet size is: 1048576, 16384